In [26]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from model import make_simulated_data, make_model, sample_model, plot_forestplots

sns.set_context('poster')
sns.set_style('white')
%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Simulation Data

In systematically measuring genotype-phenotype data, how will the data be collected, stored and analyzed? Is a small laptop enough to model the fold change calculations locally? I explore this in the following notebook.

In [27]:
# Set n_genotypes > 1000 to obtain estimates of variance in error as a function of num_measurements
# Set n_genotypes < 15 to obtain diagnostic plots (traceplot) and comparison of modelled to actual.
# You've been forewarned - for the diagnostics and modelled, plotting with > 100 takes a while...
n_genotypes = 100
n_reps = 2  # number of replicate measurements per sample

sim_data = make_simulated_data(n_genotypes=n_genotypes, n_reps=n_reps)
data, indices, num_measurements, means, sds = sim_data

In [28]:
# Compute the "actual" Z'-factor for the assay.
zp_det = 1 - (3 * sds[-2] + 3 * sds[-1]) / np.abs(means[-2] - means[-1])
zp_det

0.8439554255902848

In [29]:
model = make_model(n_genotypes, data, indices)

Applied log-transform to upper and added transformed upper_log_ to model.
Applied interval-transform to fold and added transformed fold_interval_ to model.
Applied log-transform to sigma and added transformed sigma_log_ to model.


In [30]:
%%time

trace = sample_model(model, n_genotypes)

Iteration 0 [0%]: ELBO = -946128.15
Iteration 20000 [10%]: Average ELBO = -122427.67
Iteration 40000 [20%]: Average ELBO = -1068.32
Iteration 60000 [30%]: Average ELBO = -983.39
Iteration 80000 [40%]: Average ELBO = -957.43
Iteration 100000 [50%]: Average ELBO = -951.18
Iteration 120000 [60%]: Average ELBO = -946.84
Iteration 140000 [70%]: Average ELBO = -940.35
Iteration 160000 [80%]: Average ELBO = -933.36
Iteration 180000 [90%]: Average ELBO = -930.22
Finished [100%]: Average ELBO = -928.07
CPU times: user 36.9 s, sys: 184 ms, total: 37 s
Wall time: 37.3 s


In [32]:
if n_genotypes <= 15:
    plot_forestplots(trace)

In [33]:
if n_genotypes <= 15:
    pm.summary(trace)

In [34]:
if n_genotypes <= 15:
    pm.traceplot(trace)

# Error in Estimated Mean

What is the error in estimated mean as a function of the number of measurements?

In [35]:
num_measurements

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [36]:
trace['fold'].mean(axis=0) - means

array([ -3.67455835e+00,   2.12931496e-01,  -1.39167131e+01,
         4.70389358e+00,  -5.99498289e+00,  -2.55767265e-01,
         1.37366414e-01,  -1.28481404e+00,  -2.15132752e+00,
        -2.84016935e+00,  -1.47892775e+01,  -1.22912773e+01,
        -2.15774168e+00,   7.82286723e-01,   3.57671447e+00,
        -5.92961589e+00,   3.41591480e+00,   2.81770965e+00,
        -5.78997072e+00,   5.10219290e-01,  -1.01140987e+01,
        -4.30995882e+00,   2.81125709e+00,  -2.36645582e+00,
        -1.57191483e+00,  -3.08904214e-01,  -1.84834031e-01,
         8.33206147e-01,   3.44533004e+00,   1.07924639e+00,
        -2.92910157e-01,   3.97760978e-01,   3.59805726e-01,
        -1.00793435e+01,   6.85705497e-01,  -6.44473464e+00,
        -8.68677432e-01,   1.83912738e+00,   2.94912069e-02,
        -1.17212392e+00,  -2.30296823e+00,  -2.94360717e+00,
        -1.29971587e+01,   8.92216668e+00,  -3.95183698e-01,
         3.05124458e+00,   1.73390877e+00,  -1.55848217e-01,
        -1.45733106e-02,

In [37]:
import pandas as pd
errors_df = pd.DataFrame([num_measurements, (trace['fold'].mean(axis=0) - means)]).T
errors_df.columns = ['num_measurements', 'error_means']
errors_df.head()

,num_measurements,error_means
0,2.0,-3.674558
1,2.0,0.212931
2,2.0,-13.916713
3,2.0,4.703894
4,2.0,-5.994983


In [38]:
if n_genotypes >= 500:
    sns.boxplot(data=errors_df, x='num_measurements', y='error_means')
    errors_df.groupby('num_measurements').var().plot(legend=False)
    plt.ylabel('var(error)')

In [39]:
(trace['fold'].mean(axis=0) - means).mean()

-1.2436695917196776

In [42]:
if n_genotypes <= 15:
    plt.figure()
    pm.traceplot(trace, varnames=['fold'])
    plt.figure()

# Compare modelled to actual

In [43]:
trace['fold'].mean(axis=0)

array([ 75.32544165,  62.2129315 ,  42.08328689,  67.70389358,
        24.00501711,  41.74423273,  32.13736641,  46.71518596,
        53.84867248,  71.15983065,  70.21072245,  86.7087227 ,
        63.84225832,  21.78228672,  46.57671447,  28.07038411,
        29.4159148 ,  53.81770965,  16.21002928,  90.51021929,
        81.88590128,  27.69004118,  64.81125709,  58.63354418,
        21.42808517,  22.69109579,  88.81516597,  17.83320615,
        58.44533004,  51.07924639,  68.70708984,  47.39776098,
        16.35980573,  88.92065646,  79.6857055 ,  89.55526536,
        30.13132257,  18.83912738,  75.02949121,  52.82787608,
        28.69703177,  78.05639283,  76.00284126,  24.92216668,
        63.6048163 ,  27.05124458,  51.73390877,  59.84415178,
        76.98542669,  27.90538334,  65.22103177,  37.59568435,
        88.54247588,  59.25217414,  85.69303724,  34.36212995,
        35.2991185 ,  26.04172902,  37.53009293,  45.35986087,
        56.35554001,  14.86259147,  88.35032266,  64.68

In [44]:
trace['fold'].mean(axis=0).shape

(102,)

In [45]:
np.arange(1, n_genotypes+3).shape

(102,)

In [46]:
if n_genotypes <= 15:
    fig = plt.figure()
    ax = fig.add_subplot(111)
    lower = trace['fold'].mean(axis=0) - np.percentile(trace['fold'], 2.5, axis=0)
    upper = np.percentile(trace['fold'], 97.5, axis=0) - trace['fold'].mean(axis=0)
    yerr_pos = [lower, upper]
    ax.errorbar(x=np.arange(1, n_genotypes+3), y=trace['fold'].mean(axis=0), 
                # y-error bars
                yerr=yerr_pos,
                # styling
                color='blue', ls='none', alpha=0.5, label='modeled',)
    ax.scatter(x=np.arange(1, n_genotypes+3), y=means, 
               # styling
               color=['red'] * (n_genotypes) + ['green'] + ['blue'], marker='o', s=100, label='true mean',)
    ax.errorbar(x=np.arange(1, n_genotypes+3), y=means,
               yerr=sds,
               color='red', ls='none', alpha=0.5, label='true var')
    ax.legend(loc='upper left', frameon=False)
    ax.hlines(xmin=0, xmax=n_genotypes+3, y=trace['fold'].mean(axis=0)[-2], linestyles='--', alpha=0.2)
    ax.hlines(xmin=0, xmax=n_genotypes+3, y=trace['fold'].mean(axis=0)[-1], alpha=0.2)
    ax.set_xlabel('genotype ID')
    ax.set_ylabel('fold change')
    # ax.set_xlim(, n_genotypes+2)

In [47]:
if n_genotypes <= 15:
    fig = plt.figure()
    pm.forestplot(trace, vline=1, varnames=['fold'])
print('just for checking the above plot...')

just for checking the above plot...


How often is the true mean inside the 95% HPD?

In [48]:
lower, upper = np.percentile(trace['fold'], [2.5, 97.5], axis=0)
lower, upper

(array([ 58.66078594,  55.64490455,  39.63898813,  59.40574657,
         20.206472  ,  33.43112384,  31.82636551,  45.14410895,
         49.69970741,  66.78789246,  66.15605941,  67.15038651,
         52.61618447,  17.44872481,  44.10356664,  23.02879769,
         28.30304837,  51.01745304,  12.54043065,  87.75075772,
         65.86538517,  23.89601449,  63.90646695,  56.37548887,
         21.19833438,  20.16332709,  85.00799472,  15.32515614,
         47.10587765,  48.3822574 ,  67.1742246 ,  42.58780124,
         10.89555392,  75.58725565,  78.84259489,  79.53367447,
         19.93324172,  16.31535406,  74.58478733,  49.4559607 ,
         22.12501652,  75.53665406,  70.24919032,  14.82925614,
         60.94519814,  19.08703105,  50.30028766,  50.99891725,
         76.47779185,  21.31552495,  52.21644629,  33.47181841,
         75.74977962,  56.42019101,  83.67312482,  29.03222373,
         31.37565606,  15.54964841,  31.2043672 ,  44.40934284,
         53.30432401,  10.93326007,  85.

In [49]:
means

array([ 79.        ,  62.        ,  56.        ,  63.        ,
        30.        ,  42.        ,  32.        ,  48.        ,
        56.        ,  74.        ,  85.        ,  99.        ,
        66.        ,  21.        ,  43.        ,  34.        ,
        26.        ,  51.        ,  22.        ,  90.        ,
        92.        ,  32.        ,  62.        ,  61.        ,
        23.        ,  23.        ,  89.        ,  17.        ,
        55.        ,  50.        ,  69.        ,  47.        ,
        16.        ,  99.        ,  79.        ,  96.        ,
        31.        ,  17.        ,  75.        ,  54.        ,
        31.        ,  81.        ,  89.        ,  16.        ,
        64.        ,  24.        ,  50.        ,  60.        ,
        77.        ,  26.        ,  66.        ,  38.        ,
        99.        ,  50.        ,  85.        ,  26.        ,
        35.        ,  28.        ,  41.        ,  45.        ,
        60.        ,  18.        ,  88.        ,  64.  

In [50]:
# Here is the fraction of means that have been correctly included within the 95% HPD.
sum((means > lower) *  (means < upper)) / len(means)

0.68627450980392157